# Figure 2: ER Doc Antibiotic Selection Patterns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os 
from google.cloud import bigquery

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/conorcorbin/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 
client=bigquery.Client()

In [ ]:
query = """
SELECT
    om.anon_id, om.pat_enc_csn_id_coded, om.order_med_id_coded, l.index_time, om.med_description,
    l.Ampicillin, l.Ciprofloxacin, l.Cefazolin, l.Ceftriaxone, l.Cefepime, l.Zosyn, l.Vancomycin,
    l.Meropenem, l.Vancomycin_Meropenem, l.Vancomycin_Zosyn, l.Vancomycin_Cefepime, l.Vancomycin_Ceftriaxone
FROM
    `mining-clinical-decisions.abx.abx_orders_given_and_stopped` om
INNER JOIN 
    `mining-clinical-decisions.abx.final_ast_labels` l
USING
    (pat_enc_csn_id_coded)
WHERE
    om.was_given = 1
ORDER BY 
    om.anon_id, om.pat_enc_csn_id_coded, om.order_time
"""
query_job = client.query(query)
df = query_job.result().to_dataframe()
df.head()

In [ ]:
concat_abx = lambda x : ' '.join(np.unique(sorted([a for a in x])))
(df
.assign(med_description=lambda x: [a.split(' ')[0] for a in x.med_description])
.assign(med_description=lambda x: [(a.replace('PIPERACILLIN-TAZOBACTAM-DEXTRS','PIPERACILLIN-TAZOBACTAM')
                                    .replace('VANCOMYCIN-WATER', 'VANCOMYCIN'))
                                   for a in x.med_description])
.assign(index_time=lambda x: x.index_time.dt.year)
.groupby('pat_enc_csn_id_coded')
.agg(med_description=('med_description', concat_abx),
     year=('index_time', 'first'))
.reset_index()
.groupby(['year', 'med_description'])
.agg(num_distinct_csns=('pat_enc_csn_id_coded', 'nunique'))
.reset_index()
.sort_values(['year', 'num_distinct_csns'], ascending=False)
.query("year == '2019'")
).head(20)

### For now only use CSNs with exact matches to our abx selections
Talk to Jon about this later, can we assume `PIPERACILLIN-TAZOBACTAM CEFTRIAXONE VANCOMYCIN` falls into the `PIPERACILLIN-TAZOBACTAM VANCOMYCIN` category for instance. 

In [ ]:
from tqdm import tqdm
def compute_was_covered(x, random=False):
    """
    Given med description, find appropriate label column and return whether patient was covered during CSN
    Returns "Not in abx options" if abx regimen isn't in our set of 12 options - useful for filtering later
    """
    if not random:
        med_description = x.med_description
    else:
        med_description = x.random_med_description
        
    if med_description == "CEFTRIAXONE":
        return x.Ceftriaxone
    elif med_description == "PIPERACILLIN-TAZOBACTAM VANCOMYCIN":
        return x.Vancomycin_Zosyn
    elif med_description == "PIPERACILLIN-TAZOBACTAM":
        return x.Zosyn
    elif med_description == "CEFTRIAXONE VANCOMYCIN":
        return x.Vancomycin_Ceftriaxone
    elif med_description == "CEFEPIME VANCOMYCIN":
        return x.Vancomycin_Cefepime
    elif med_description == "CEFEPIME":
        return x.Cefepime
    elif med_description == "VANCOMYCIN":
        return x.Vancomycin
    elif med_description == "MEROPENEM":
        return x.Meropenem
    elif med_description == "MEROPENEM VANCOMYCIN":
        return x.Vancomycin_Meropenem
    elif med_description == "CEFAZOLIN":
        return x.Cefazolin
    elif med_description == "CIPROFLOXACIN":
        return x.Ciprofloxacin
    elif med_description == "AMPICILLIN":
        return x.Ampicillin
    else:
        return "Not in abx options"

df_final = pd.DataFrame()
clin_covered_rates = []
random_covered_rates = []
for i in tqdm(range(100)):
    # Aggregate clinician orders
    df_drugs = (df
        .assign(med_description=lambda x: [a.split(' ')[0] for a in x.med_description])
        .assign(med_description=lambda x: [(a.replace('PIPERACILLIN-TAZOBACTAM-DEXTRS','PIPERACILLIN-TAZOBACTAM')
                                            .replace('VANCOMYCIN-WATER', 'VANCOMYCIN'))
                                           for a in x.med_description])
        .assign(index_time=lambda x: x.index_time.dt.year)
        .groupby('pat_enc_csn_id_coded')
        .agg({'med_description' : concat_abx,
              'index_time' : 'first',
              'Ampicillin' : 'first',
              'Ciprofloxacin' : 'first',
              'Cefazolin' : 'first',
              'Ceftriaxone' : 'first',
              'Cefepime' : 'first',
              'Zosyn' : 'first',
              'Vancomycin' : 'first',
              'Meropenem' : 'first',
              'Vancomycin_Ceftriaxone' : 'first',
              'Vancomycin_Cefepime' : 'first',
              'Vancomycin_Zosyn' : 'first',
              'Vancomycin_Meropenem' : 'first'})
        .rename(columns={'index_time' : 'year'})
        .reset_index()    
    )

    # Create flag for whether clinicians covered the patient during the csn, then filter for 
    df_drugs = (df_drugs
        .query("year == 2019")
        .sample(frac=1.0, replace=True) # bootstrap each iteration
        .assign(was_covered_dr=df_drugs.apply(lambda x: compute_was_covered(x), axis=1))
        .query("was_covered_dr == 1 or was_covered_dr == 0")
        .assign(random_med_description=lambda x: np.random.choice(x.med_description, size=len(x.med_description), replace=False))
    )

    # create a column for a randomly shuffled med description and then compute when random assignemnt covers patient
    df_drugs = (df_drugs
        .assign(was_covered_random=df_drugs.apply(lambda x: compute_was_covered(x, random=True), axis=1))
        .groupby(['med_description'])
        .agg(num_distinct_csns=('pat_enc_csn_id_coded', 'count'),
             num_times_covered_dr=('was_covered_dr', 'sum'),
             num_times_covered_random=('was_covered_random', 'sum'))
        .reset_index()
        .sort_values(['num_distinct_csns'], ascending=False)
    )
    
    clin_covered_rate = df_drugs['num_times_covered_dr'].sum() / df_drugs['num_distinct_csns'].sum()
    random_covered_rate = df_drugs['num_times_covered_random'].sum() / df_drugs['num_distinct_csns'].sum()
    clin_covered_rates.append(clin_covered_rate)
    random_covered_rates.append(random_covered_rate)
    
    df_final = pd.concat([df_final, df_drugs])



In [ ]:
print("Clinician Coverage Rate:{:.2f} [{:.2f}-{:.2f}]".format(np.mean(clin_covered_rates),
                                                              np.percentile(clin_covered_rates, 2.5),
                                                              np.percentile(clin_covered_rates, 97.5))
     )

print("Random Coverage Rate:{:.2f} [{:.2f}-{:.2f}]".format(np.mean(random_covered_rates),
                                                           np.percentile(random_covered_rates, 2.5),
                                                           np.percentile(random_covered_rates, 97.5))
     )


### Figure 2 Plot

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
sns.set(style='white', font_scale=1.5)
df_final = df_final.sort_values('med_description')
sns.barplot(x="num_distinct_csns", y="med_description", ci='sd', data=df_final, ax=ax, color='red' )
sns.barplot(x="num_times_covered_dr", y="med_description", ci=None, data=df_final, ax=ax, color='blue')
ax.set_ylabel("")
ax.set_xlabel("ER Encounters")
ax.set_title("Clinician Antibiotic Selections and Coverage")
plt.show()

In [ ]:
# Plot
import pdb
year_sets = [["2009", "2010", "2011", "2012"],
             ["2013", "2014", "2015", "2016"],
             ["2017", "2018", "2019"]]

fig, ax = plt.subplots(3, 1, figsize=(8, 24))
sns.set(style='white', font_scale=1.5)
row, col = 0, 0
for i, ys in enumerate(year_sets):
    df_year = (df_drugs
        .query("year in @ys", engine='python')
        .groupby("med_description")
        .agg({'num_distinct_csns' : 'sum',
              'num_times_covered' : 'sum'})
        .reset_index()
    )
    sns.barplot(x="num_distinct_csns", y="med_description", data=df_year, ax=ax[i], color='red')
    sns.barplot(x="num_times_covered", y="med_description", data=df_year, ax=ax[i], color='blue')
    ax[i].set_ylabel("")
    ax[i].set_xlabel("ER Encounters")
    ax[i].set_title("Clinician Antibiotic Selections %s-%s" % (ys[0], ys[-1]))
plt.show()

In [ ]:
### 